In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
.appName("Test Transformation Script") \
.getOrCreate()


24/08/30 22:54:51 WARN Utils: Your hostname, codespaces-ef2dca resolves to a loopback address: 127.0.0.1; using 10.0.1.169 instead (on interface eth0)
24/08/30 22:54:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/30 22:54:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark)

In [4]:
df = spark.read.json("issues.json")

In [5]:
df.printSchema()
from pyspark.sql.functions import col, to_timestamp

root
 |-- acknowledged_at: string (nullable = true)
 |-- address: string (nullable = true)
 |-- closed_at: string (nullable = true)
 |-- comment_url: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- description: string (nullable = true)
 |-- flag_url: string (nullable = true)
 |-- html_url: string (nullable = true)
 |-- id: long (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- media: struct (nullable = true)
 |    |-- image_full: string (nullable = true)
 |    |-- image_square_100x100: string (nullable = true)
 |    |-- representative_image_url: string (nullable = true)
 |    |-- video_url: string (nullable = true)
 |-- point: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- private_visibility: boolean (nullable = true)
 |-- rating: long (nullable = true)
 |-- reopened_at: string (nullable = true)
 |

In [6]:
def select_col(df):
    selected_df = df.select(
        col("id"),
        col("reporter.name"),
        col("created_at"),
        col("updated_at"),
        col("status"),
        col("summary"),
        col("description"),
        col("lat"),
        col("lng"),
        col("address"),
        col("url"),
        col("reporter.role"),
        col("request_type.organization"),
        col("request_type.title"),
        col("request_type.url"),
        col("private_visibility"),
        col("rating")
    )
    
    return selected_df

In [7]:
df = select_col(df)
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- status: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- description: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- address: string (nullable = true)
 |-- url: string (nullable = true)
 |-- role: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- private_visibility: boolean (nullable = true)
 |-- rating: long (nullable = true)



In [8]:
df.printSchema()
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- status: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- description: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- address: string (nullable = true)
 |-- url: string (nullable = true)
 |-- role: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- private_visibility: boolean (nullable = true)
 |-- rating: long (nullable = true)



In [9]:
def rename_columns(df: DataFrame) -> DataFrame:

    r_dict = {
        "id": "id",
        "name": "reporter_name",
        "created_at": "issue_created_at",
        "updated_at": "issue_updated_at",
        "status": "issue_status",
        "summary": "issue_summary",
        "description": "issue_description",
        "lat": "latitude",
        "lng": "longitude",
        "address": "issue_address",
        "url": "url",
        "role": "reporter_role",
        "organization": "organization",
        "title": "title",
        "private_visibility": "is_private",
        "rating": "rating"
    }

    for old_name, new_name in r_dict.items():
        df = df.withColumnRenamed(old_name, new_name)
    
    return df

In [10]:
df1 = rename_columns(df)

In [11]:
df1.printSchema()

root
 |-- id: long (nullable = true)
 |-- reporter_name: string (nullable = true)
 |-- issue_created_at: string (nullable = true)
 |-- issue_updated_at: string (nullable = true)
 |-- issue_status: string (nullable = true)
 |-- issue_summary: string (nullable = true)
 |-- issue_description: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- issue_address: string (nullable = true)
 |-- url: string (nullable = true)
 |-- reporter_role: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_private: boolean (nullable = true)
 |-- rating: long (nullable = true)



In [12]:
df1.show(1)

+--------+-------------+--------------------+--------------------+------------+--------------------+--------------------+----------------+-----------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+----------+------+
|      id|reporter_name|    issue_created_at|    issue_updated_at|issue_status|       issue_summary|   issue_description|        latitude|        longitude|       issue_address|                 url|    reporter_role|        organization|               title|                 url|is_private|rating|
+--------+-------------+--------------------+--------------------+------------+--------------------+--------------------+----------------+-----------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+----------+------+
|17398644|      311_MDP|2024-08-29T11:56:...|2024-08-29T11:56:...|        Open|Inoperable Vehicl...|citize

In [13]:
from pyspark.sql.functions import col, to_timestamp, to_date
from pyspark.sql.types import LongType, StringType, DoubleType, BooleanType

def convert_data_types(df: DataFrame) -> DataFrame:
    # Convert to appropriate data types
    df = df.withColumn("created_at", to_timestamp(col("created_at"))) \
           .withColumn("updated_at", to_timestamp(col("updated_at")))

    return df



In [14]:
df2 = df1.withColumnRenamed(df1.columns[9], "issue_url")
df2.printSchema()
df2 = convert_data_types(df2)
df2.show(1)

root
 |-- id: long (nullable = true)
 |-- reporter_name: string (nullable = true)
 |-- issue_created_at: string (nullable = true)
 |-- issue_updated_at: string (nullable = true)
 |-- issue_status: string (nullable = true)
 |-- issue_summary: string (nullable = true)
 |-- issue_description: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- issue_url: string (nullable = true)
 |-- url: string (nullable = true)
 |-- reporter_role: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- is_private: boolean (nullable = true)
 |-- rating: long (nullable = true)



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `created_at` cannot be resolved. Did you mean one of the following? [`rating`, `issue_created_at`, `id`, `latitude`, `reporter_name`].;
'Project [id#79L, reporter_name#97, issue_created_at#115, issue_updated_at#133, issue_status#151, issue_summary#169, issue_description#187, latitude#205, longitude#223, issue_url#451, url#259, reporter_role#278, organization#296, title#314, url#260, is_private#332, rating#350L, to_timestamp('created_at, None, TimestampType, Some(Etc/UTC), false) AS created_at#469]
+- Project [id#79L, reporter_name#97, issue_created_at#115, issue_updated_at#133, issue_status#151, issue_summary#169, issue_description#187, latitude#205, longitude#223, issue_address#241 AS issue_url#451, url#259, reporter_role#278, organization#296, title#314, url#260, is_private#332, rating#350L]
   +- Project [id#79L, reporter_name#97, issue_created_at#115, issue_updated_at#133, issue_status#1

24/08/30 22:55:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
df2.printSchema()
from pyspark.sql.functions import col, isnan, when, count

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- description: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- issue_url: string (nullable = true)
 |-- reporter_url: string (nullable = true)
 |-- role: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- title: string (nullable = true)
 |-- reporter_url: string (nullable = true)
 |-- private_visibility: boolean (nullable = true)
 |-- rating: long (nullable = true)



In [ ]:
df = df2.drop("reporter_url")

In [ ]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- updated_at: timestamp (nullable = true)
 |-- status: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- description: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- issue_url: string (nullable = true)
 |-- role: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- title: string (nullable = true)
 |-- private_visibility: boolean (nullable = true)
 |-- rating: long (nullable = true)



In [ ]:
df.show(2)

+--------+-------------------+-------------------+-------------------+------+--------------------+--------------------+----------------+-----------------+--------------------+-----------------+--------------------+--------------------+------------------+------+
|      id|               name|         created_at|         updated_at|status|             summary|         description|             lat|              lng|           issue_url|             role|        organization|               title|private_visibility|rating|
+--------+-------------------+-------------------+-------------------+------+--------------------+--------------------+----------------+-----------------+--------------------+-----------------+--------------------+--------------------+------------------+------+
|17398644|            311_MDP|2024-08-29 15:56:08|2024-08-29 15:56:09|  Open|Inoperable Vehicl...|citizen states th...|33.4828359940456|-86.8150319842692|604 22nd Ave S Bi...|Verified Official|Birmingham Police...|

In [ ]:
from pyspark.sql.functions import sum

In [ ]:
missing_values_count = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
missing_values_count.show()

+---+----+----------+----------+------+-------+-----------+---+---+---------+----+------------+-----+------------------+------+
| id|name|created_at|updated_at|status|summary|description|lat|lng|issue_url|role|organization|title|private_visibility|rating|
+---+----+----------+----------+------+-------+-----------+---+---+---------+----+------------+-----+------------------+------+
|  0|   0|         0|         0|     0|      0|          0|  0|  0|        0|   0|           0|    0|                 0|     0|
+---+----+----------+----------+------+-------+-----------+---+---+---------+----+------------+-----+------------------+------+



AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "element_at(summary, summary)" due to data type mismatch: Parameter 1 requires the ("ARRAY" or "MAP") type, however "summary" has the type "STRING".;
'Project [summary#4995, element_at(id#4710, summary#4995, None, false) AS id#4766, element_at(name#4715, summary#4995, None, false) AS name#4767, element_at(status#4720, summary#4995, None, false) AS status#4768, element_at(summary#4995, summary#4995, None, false) AS summary#4769, element_at(description#4730, summary#4995, None, false) AS description#4770, element_at(lat#4735, summary#4995, None, false) AS lat#4771, element_at(lng#4740, summary#4995, None, false) AS lng#4772, element_at(issue_url#4745, summary#4995, None, false) AS issue_url#4773, element_at(role#4750, summary#4995, None, false) AS role#4774, element_at(organization#4755, summary#4995, None, false) AS organization#4775, element_at(title#4760, summary#4995, None, false) AS title#4776, element_at(ra